In [130]:
import pandas as pd

from util import Pipeline

p = Pipeline('configs')

In [131]:
ofm_prev = p.get_table('ofm_estimates_prev_year_by_control_area')

In [132]:
ofm_dec = p.get_table('ofm_estimates_census_year_by_control_area')

In [133]:
dec = p.get_table('decennial_by_control_area')

In [134]:
df = dec.merge(ofm_prev, on='control_id',how='outer').merge(ofm_dec, on='control_id',how='outer',suffixes=('_prev_year','_census_year'))

In [135]:
def combine_targets(pipeline):
    df = pd.DataFrame()
    for table in pipeline.settings['targets_tables']:
        if 'pop_chg_col' in table:
            # only load targets tables that use population change (Kitsap, Pierce and Snohomish)
            df_table = pipeline.get_table(table['name'])
            
            # add start and horizon year columns
            df_table['start'] = table['pop_chg_start']
            df_table['horizon'] = table['pop_chg_horizon']
            
            # combine into single dataframe
            df = pd.concat([df, df_table], ignore_index=True)
    
    return df[['target_id','pop_chg','start','horizon']]

targets = combine_targets(p)

In [137]:
control_target_xwalk = p.get_table('control_target_lookup')

In [138]:
df = df.merge(control_target_xwalk[['control_id','target_id']], on='control_id', how='left').groupby('target_id').sum().drop(columns=['control_id']).reset_index()

In [142]:
targets = targets.merge(df, on='target_id', how='left')